# LlamaIndex を使ってみる

参照

- https://docs.llamaindex.ai/en/stable/getting_started/customization.html


試すときに使うデータの取得。clipper というライブラリで HTML を Markdown 化します。
今回 npm を使ってインストールしますが、BeautifulSoup などでも似たことができると思います。


In [7]:
!npm install @philschmid/clipper

!mkdir -p ./data
!cd ./data
!clipper clip -u https://ameblo.jp/ca-seo/entry-12835366312.html -o ./data/ca-seo-2024.md

事前に OPENAI_API_KEY をセットアップ。

`!export OPENAI_API_KEY=XXXXX`


In [2]:
!pip install llama-index -Uqq

In [8]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [9]:
documents

[Document(id_='fdf3c3d2-22ef-4cb5-a80c-d6b540baf0a4', embedding=None, metadata={'file_path': 'data/ca-seo-2024.md', 'file_name': 'ca-seo-2024.md', 'file_type': None, 'file_size': 9177, 'creation_date': '2024-01-09', 'last_modified_date': '2024-01-09', 'last_accessed_date': '2024-01-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='f098789fcb17e5cd0085d104d17dc6eac6140dad5dc479281b0c1e8576530a6f', text='\n\n2024年 SEOと検索エンジンの予想\n\nここ2年ほど年初にその年の検索エンジンやSEOについて勝手に予想してきましたが、せっかくなのでアメブロ全体のアクセスにちょびっと貢献する意味も込めて今年もやってみたいと思います。\n\n2022年\u3000\n\n2023年\n\n2024年は元旦から自然災害が発生するという誰も予想していなかったスタートとなりました。\n\n改めて被災者の方に心よりお見舞い申し上げるとともに犠牲者の方のご冥福をお祈り申し上げます。\n\nさて、2024年のSEO・検索エンジンについての予想ですが結論からいうと2023年の流れのままであろうと思います。\n\n1.アルゴリズムは信頼性重視になる\n\n昨日、ちょうど辻さんとも話していましたが、アメリカで11月には

In [10]:
index

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("SGEとはなんですか？")
print(response)

SGEはSearch Generative Experienceの略であり、Googleの検索エンジンに関連する技術や機能の一部を指します。現在、SGEはSearch Labs経由での提供となっており、一般には広く公開されていません。SGEは、大規模言語モデルの進化やハルシネーションへの対策など、Googleの検索エンジンの進化に関連していると考えられます。ただし、2024年の早い段階で通常のGoogle検索に実装されるかどうかは予測が分かれており、社会情勢の不安定さや競合他社の動向なども考慮される必要があります。


In [12]:
# store embeddings to file
index.storage_context.persist("./storage")

In [13]:
# load the existing index
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)

### チャンクサイズを変えたい場合


In [22]:
# chunk in smaller chunks
from llama_index import ServiceContext
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# デフォルトが1024なので半分で試してみます。
service_context = ServiceContext.from_defaults(chunk_size=512)

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("SGEとはなんですか？")
print(response)

SGE stands for Search Generative Experience. It is a feature provided by Google that is currently only available through Search Labs and is not widely accessible to the general public. SGE is a part of Google's evolving search experience and is expected to continue to advance, with improvements in large-scale language models and measures to address hallucination. However, it is unlikely to be launched for regular Google searches in the near future.


### ベクターストアを変えたい場合


In [23]:
# using Chroma for embeddings
!pip install chromadb -Uqq
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [24]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
query_engine = index.as_query_engine()
response = query_engine.query("SGEとはなんですか？")
print(response)

SGEはSearch Generative Experienceの略であり、Googleの検索エンジンに関連する技術や機能の一部を指します。現在、SGEはSearch Labs経由での提供となっており、一般に広く公開されているわけではありません。SGEは、大規模言語モデルの進化やハルシネーションへの対策など、Googleの検索エンジンの進化に関連していると考えられます。ただし、2024年の早い段階で通常のGoogle検索にSGEが実装されるかどうかは予測できません。


### コンテキストに使うチャンクの数を変えたい場合


In [25]:
query_engine = index.as_query_engine(similarity_top_k=5)
query_engine.query("SGEとはなんですか？")

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Response(response="SGE stands for Search Generative Experience. It is a feature or experience provided by Google that is currently only available through Search Labs and not widely accessible to the general public. It is a part of Google's ongoing efforts to evolve and improve its search experience using large-scale language models like Bard. While some SEO experts speculate that SGE may be implemented in regular Google search by the end of 2024, it is unlikely to be released earlier due to concerns about the risk of generating harmful or misleading content. Additionally, SGE was originally developed as a response to Microsoft's Bard, but Google may not see the need to take on the risk of releasing SGE, especially considering that Bing's usage rate has not significantly increased.", source_nodes=[NodeWithScore(node=TextNode(id_='3fe29053-3523-476b-8892-24f4a1255d5e', embedding=None, metadata={'file_path': 'data/ca-seo-2024.md', 'file_name': 'ca-seo-2024.md', 'file_type': None, 'file_si

In [26]:
query_engine = index.as_query_engine(similarity_top_k=5)
query_engine.query("必ず日本語で答えてください。SGEとはなんですか？")

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Response(response='SGEとは、Search Generative Experienceの略であり、Googleの検索エンジンにおける生成系AIの一部を指します。現在、SGEはSearch Labs経由での提供となっており、一般に広く公開されているわけではありません。SGEは、大規模言語モデルの進化やハルシネーションへの対策など、Googleの検索エンジンの進化に関連しています。ただし、SGEが2024年の早い段階で通常のGoogle検索に実装されるかどうかは予測が分かれており、具体的な実装時期は不明です。', source_nodes=[NodeWithScore(node=TextNode(id_='3fe29053-3523-476b-8892-24f4a1255d5e', embedding=None, metadata={'file_path': 'data/ca-seo-2024.md', 'file_name': 'ca-seo-2024.md', 'file_type': None, 'file_size': 9177, 'creation_date': '2024-01-09', 'last_modified_date': '2024-01-09', 'last_accessed_date': '2024-01-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='73f38ee2-e721-4473-aece-f1136efcd30c', node_type=<ObjectType.DOCUMENT: '4'>, me

### 違う LLM を使いたい場合


In [ ]:
from llama_index import ServiceContext
from llama_index.llms import PaLM

service_context = ServiceContext.from_defaults(llm=PaLM())

### レスポンスの生成方法を変えたい場合

- https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes.html


In [28]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(response_mode="tree_summarize")
response = query_engine.query("必ず日本語で答えてください。SGEとはなんですか？")
print(response)

SGEとは、GoogleのSearch Generative Experienceの略称です。現在、SGEはSearch Labs経由での提供となっており、一般に広く公開されていません。SGEは、大規模言語モデルの一種であり、Googleの検索体験をさらに進化させることが期待されています。ただし、SGEが通常のGoogle検索に実装される時期については予測が分かれており、2024年の早い段階で実装されると予想する専門家もいれば、年末まで実装されないと予想する専門家もいます。


### ストリーミングをオンにする


In [31]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
# index = VectorStoreIndex.from_documents(documents)
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine(streaming=True)
response = query_engine.query(
    "必ず日本語で答えてください。\n必ずドキュメントをソースとした解答を述べ、ソースとなった文章を最後にリストアップしてください。\nGoogleは収益化のためにどんなことをしそうですか？"
)
response.print_response_stream()

Googleは検索結果の途中にリスティング広告を差し込んだり、動画検索をYouTubeに特化させたりするなど、収益化戦略を加速させています。さらに、Googleは広告の増加や資産への誘導など、サイト運営者やSEOsにとってはつらい要素もあります。この流れは今後も止まらず、さまざまなバーティカルサーチにおいてビジネス展開する可能性も考えられます。

ソース:
- file_path: data/ca-seo-2024.md
- Context: 3.Googleの収益戦略の加速

### ドキュメント QA ではなく、チャットボットモードにしたい場合


In [36]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

service_context = ServiceContext.from_defaults(chunk_size=512)
storage_context = StorageContext.from_defaults(persist_dir="./storage")

documents = SimpleDirectoryReader("data").load_data()
index = load_index_from_storage(storage_context, service_context=service_context)

query_engine = index.as_chat_engine(streaming=True)
response = query_engine.chat(
    "必ず日本語で答えてください。\n必ずドキュメントをソースとした解答を述べ、ソースとなった文章を最後にリストアップしてください。\nGoogleは収益化のためにどんなことをしそうですか？"
)
print(response)

Googleは収益化のために以下のようなことを行っていると考えられます。

1. リスティング広告の提供: Googleは検索結果ページにリスティング広告を表示することで収益を得ています。これにより、企業は自社の広告を特定のキーワードで表示させることができます。

2. 動画広告の増加: YouTubeなどの動画プラットフォームでの広告表示を増やすことで収益を増やしています。動画広告は視聴者の注意を引きやすく、広告主にとって効果的な広告手段となっています。

3. バーティカルサーチへの進出: Googleはさまざまな分野に進出し、特定の業界やカテゴリに特化した検索エンジンを提供することで収益を増やす可能性があります。例えば、ホテル予約や商品比較などの分野でのバーティカルサーチが考えられます。

これらの戦略により、Googleは広告収益を最大化し、収益化を図っています。

ソース:
- [Googleの収益化戦略](https://www.businessinsider.jp/post-228762)


In [37]:
response = query_engine.chat("「動画広告の増加」について、他にはどのようなことが書かれていますか？")
print(response)

Googleの動画広告増加の戦略に関して、以下のような情報があります。

1. 動画検索の増加: Googleは動画検索を強化し、YouTubeを中心に動画コンテンツを増やしています。これにより、ユーザーが動画を検索しやすくなり、広告主はより多くの視聴者にアプローチする機会を得ることができます。

2. YouTubeでの広告増加: YouTubeはGoogleの傘下にある動画プラットフォームであり、広告表示が増えています。YouTubeは視聴者数が多く、広告主にとって魅力的な広告媒体となっています。

3. リスティング広告の継続: Googleはリスティング広告を重要な収益源として位置付けており、動画広告の増加によってもリスティング広告の提供は減らされることはないと予想されます。

Googleは動画広告の増加を通じて収益を増やす一方で、ウェブサイト運営者やSEOにとっては競争が激化し、厳しい状況となっているようです。

ソース:
- [Googleの収益化戦略](https://www.businessinsider.jp/post-228762)
